In [1]:
# 1. Importar librerías
import earthaccess
import os
import xarray as xr
import pandas as pd

C:\Users\leona\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 2. Login interactivo (EDL)
auth = earthaccess.login(strategy="interactive")
print("Authenticated?", auth.authenticated)

Authenticated? True


In [3]:
# 3. Parámetros de búsqueda
shortname = "TEMPO_NO2_L3"  # ✅ Dataset correcto
date_start = "2025-09-20 00:00:00"
date_end   = "2025-09-28 23:59:59"

print(f"\nSearching granules for {shortname}...")


Searching granules for TEMPO_NO2_L3...


In [4]:
# 4. Buscar granules
granules = earthaccess.search_data(
    short_name=shortname,
    temporal=(date_start, date_end)
)

print(f"Found {len(granules)} granules")

if not granules:
    raise ValueError("❌ No se encontraron granules en este rango, intenta ampliar fechas")

Found 108 granules


In [5]:
# 5. Descargar un granule al azar
download_dir = "./tempo_data"
os.makedirs(download_dir, exist_ok=True)

local_paths = earthaccess.download(
    granules=[granules[0]],  # baja solo el primero para pruebas
    local_path=download_dir
)

print("✅ Granule descargado:", local_paths[0])


QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 124.07it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]

✅ Granule descargado: tempo_data\TEMPO_NO2_L3_V04_20250920T114128Z_S002.nc


In [6]:
# 6. Abrir con xarray
ds = xr.open_dataset(local_paths[0])
print(ds)


<xarray.Dataset> Size: 91MB
Dimensions:    (latitude: 2950, longitude: 7750, time: 1)
Coordinates:
  * longitude  (longitude) float32 31kB -168.0 -168.0 -167.9 ... -13.03 -13.01
  * latitude   (latitude) float32 12kB 14.01 14.03 14.05 ... 72.95 72.97 72.99
  * time       (time) datetime64[ns] 8B 2025-09-20T11:41:46.026923776
Data variables:
    weight     (latitude, longitude) float32 91MB ...
Attributes: (12/40)
    history:                          2025-09-20T15:12:36Z: L2_regrid -v /tem...
    scan_num:                         2
    time_coverage_start:              2025-09-20T11:41:28Z
    time_coverage_end:                2025-09-20T12:21:17Z
    time_coverage_start_since_epoch:  1442403706.0269237
    time_coverage_end_since_epoch:    1442406095.6548083
    ...                               ...
    title:                            TEMPO Level 3 nitrogen dioxide product
    collection_shortname:             TEMPO_NO2_L3
    collection_version:               1
    keywords:       

In [ ]:
#Por si se ocupan revisar las variables acceso rápido:
#print("Variables en este granule:", list(ds.variables))


Variables en este granule: ['weight', 'longitude', 'latitude', 'time']


In [ ]:
 # 7. Explorar variables disponibles
print("Variables en este granule:", list(ds.variables))

# Buscar automáticamente variables que contengan "NO2" en su nombre
no2_vars = [v for v in ds.variables if "NO2" in v.upper()]
print("Variables candidatas de NO2:", no2_vars)

# 8. Si encontramos al menos una, exportamos la primera
if no2_vars:
    var = no2_vars[0]  # agarramos la primera candidata
    print(f"\n✅ Usando variable: {var}")

    df = ds[[var]].to_dataframe().reset_index()
    df.to_csv("tempo_no2_sample.csv", index=False)
    df.to_parquet("tempo_no2_sample.parquet", index=False)
    print("✅ Data exportada a CSV y Parquet")
else:
    print("⚠ No se encontró ninguna variable con 'NO2' en el nombre")

Variables en este granule: ['weight', 'longitude', 'latitude', 'time']
Variables candidatas de NO2: []
⚠ No se encontró ninguna variable con 'NO2' en el nombre
